In [1]:
## import necessary libraries 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
pd.set_option('display.max_columns', 500)
warnings.filterwarnings('ignore')


In [2]:
## upload books dataset

books_df = pd.read_csv('books.csv')
books_df.head()

id  book_id  best_book_id  work_id  books_count       isbn        isbn13  \
0   1  2767052       2767052  2792775          272  439023483  9.780439e+12   
1   2        3             3  4640799          491  439554934  9.780440e+12   
2   3    41865         41865  3212258          226  316015849  9.780316e+12   
3   4     2657          2657  3275794          487   61120081  9.780061e+12   
4   5     4671          4671   245494         1356  743273567  9.780743e+12   

                       authors  original_publication_year  \
0              Suzanne Collins                     2008.0   
1  J.K. Rowling, Mary GrandPré                     1997.0   
2              Stephenie Meyer                     2005.0   
3                   Harper Lee                     1960.0   
4          F. Scott Fitzgerald                     1925.0   

                             original_title  \
0                          The Hunger Games   
1  Harry Potter and the Philosopher's Stone   
2                                  Twilight   
3                     To Kill a Mockingbird   
4                          The Great Gatsby   

                                               title language_code  \
0            The Hunger Games (The Hunger Games, #1)           eng   
1  Harry Potter and the Sorcerer's Stone (Harry P...           eng   
2                            Twilight (Twilight, #1)         en-US   
3                              To Kill a Mockingbird           eng   
4                                   The Great Gatsby           eng   

   average_rating  ratings_count  work_ratings_count  work_text_reviews_count  \
0            4.34        4780653             4942365                   155254   
1            4.44        4602479             4800065                    75867   
2            3.57        3866839             3916824                    95009   
3            4.25        3198671             3340896                    72586   
4            3.89        2683664             2773745                    51992   

   ratings_1  ratings_2  ratings_3  ratings_4  ratings_5  \
0      66715     127936     560092    1481305    2706317   
1      75504     101676     455024    1156318    3011543   
2     456191     436802     793319     875073    1355439   
3      60427     117415     446835    1001952    1714267   
4      86236     197621     606158     936012     947718   

                                           image_url  \
0  https://images.gr-assets.com/books/1447303603m...   
1  https://images.gr-assets.com/books/1474154022m...   
2  https://images.gr-assets.com/books/1361039443m...   
3  https://images.gr-assets.com/books/1361975680m...   
4  https://images.gr-assets.com/books/1490528560m...   

                                     small_image_url  
0  https://images.gr-assets.com/books/1447303603s...  
1  https://images.gr-assets.com/books/1474154022s...  
2  https://images.gr-assets.com/books/1361039443s...  
3  https://images.gr-assets.com/books/1361975680s...  
4  https://images.gr-assets.com/books/1490528560s...

In [3]:
## import rating dataset

rating = pd.read_csv('ratings.csv')
rating.head()

book_id  user_id  rating
0        1      314       5
1        1      439       3
2        1      588       5
3        1     1169       4
4        1     1185       4

In [4]:
## select a random user_id

random_user_id = rating['user_id'].sample(random_state=15).values[0]
random_user_id

12455

In [5]:
def hybrid_recommend(books_df, rating_df, user_id):
    
    # USER BASED RECOMMENDATION
    
    # Merge books and rating df
    books = books_df[['id', 'authors', 'title', 'ratings_count']]
    books.rename(columns={"id":"book_id"}, inplace=True)
    df = rating.merge(books, on='book_id', how='left')
    
    # Creating user_books_df
    users_books_df = df.pivot_table(index='user_id', columns='title', values='rating')
    
    
    # Random user's books
    random_user_df = users_books_df[users_books_df.index == user_id]
    books_read_by_random_user = df[df['user_id'] == user_id]['title'].value_counts().index.tolist()
    
    # Other users who read similar books
    books_read_df = users_books_df[books_read_by_random_user]
    same_books_counter = books_read_df.notnull().sum(axis=1).reset_index()
    same_books_counter.columns = ['user_id', 'books_count']
    same_books_counter = same_books_counter[same_books_counter['books_count'] > 0].sort_values('books_count', ascending=False)
    perc = len(books_read_by_random_user) * 10 / 100
    users_read_same_books = same_books_counter[same_books_counter['books_count'] > perc]
    same_user_list = users_read_same_books['user_id'].tolist()[1:]
    
    # Calculate correlations
    concat_df = pd.concat([books_read_df[books_read_df.index.isin(same_user_list)],
                      books_read_df[books_read_df.index == user_id]])
    corr_df = concat_df.T.corr().unstack().sort_values().drop_duplicates()
    corr_df = pd.DataFrame(corr_df, columns=['corr'])
    corr_df.index.names = ['user_id_1', 'user_id_2']
    corr_df.reset_index(inplace=True)
    corr_df[(corr_df['user_id_1'] == user_id) & (corr_df['corr'] > 0.50)].sort_values('corr', ascending=False)
    
    # Make recommendations
    top_users = corr_df[(corr_df['user_id_1'] == user_id) & (corr_df['corr'] > 0.65)].sort_values('corr', ascending=False)[['user_id_2', 'corr']]
    top_users.rename(columns={'user_id_2': 'user_id'}, inplace=True)
    top_users_rating = top_users.merge(rating, how='inner')
    top_users_rating['weighted_average'] = top_users_rating['corr'] * top_users_rating['rating']
    top_users_rating = top_users_rating.sort_values('weighted_average', ascending=False)
    recommendation_df = top_users_rating.groupby('book_id').agg({'weighted_average': 'mean'}).reset_index()
    recommendation_df = recommendation_df[recommendation_df['weighted_average'] > 4][0:5]
    books_to_be_recommended = books[books['book_id'].isin(recommendation_df['book_id'].tolist())]
    user_based_5_books = books_to_be_recommended['title'].tolist()
    
    # ITEM BASED RECOMMENDATION
    
    # The most popular book that random user given 5 points
    books_given_5_points = df[(df['user_id'] == user_id) & (df['rating'] == 5)]
    most_popular_book = books_given_5_points[books_given_5_points['ratings_count'] == books_given_5_points['ratings_count'].max()]
    most_popular_book = most_popular_book['title'].values[0]
    
    # Make recommendations
    users_books_df.corrwith(users_books_df[most_popular_book]).sort_values(ascending=False).head(5)
    item_based_5_books = users_books_df.corrwith(users_books_df[most_popular_book]).sort_values(ascending=False).head(5).index.tolist()
    
    
    # Concatenate user and item based list and delete dublicates
    recommend_10_books = [*user_based_5_books, *item_based_5_books] 
    recommend_10_books = list(set(recommend_10_books))
    
    return recommend_10_books

In [6]:
hybrid_recommend(books_df, rating, user_id=random_user_id)

['Pale Fire',
 'How to Be Good',
 "I'd Tell You I Love You, But Then I'd Have to Kill You (Gallagher Girls, #1)",
 'Mirror Mirror',
 'Infidel']

In [7]:
### Get a random user and make recommendation

random_user_id_2 = rating['user_id'].sample().values[0]
random_user_id_2

21842

In [8]:
## make recommendations

hybrid_recommend(books_df, rating, user_id=random_user_id_2)

['Jewels of the Sun (Gallaghers of Ardmore, #1)',
 'The Boxcar Children (The Boxcar Children, #1)',
 'Sacred (Kenzie & Gennaro, #3)',
 "The Shelters of Stone (Earth's Children, #5)",
 'The Last Coyote (Harry Bosch, #4; Harry Bosch Universe, #4)']